# Dataset description

The data represents the energy values of supramolecular systems, which were calculated using two different quantum chemical approximations. The "HF" (Hartree-Fock) set was calculated using a fast and inaccurate approximation, while "DFT" (Density Functional Theory) was calculated using a resource-intensive but accurate approximation.

Feature  | Feature Type | Description
-------------------|--------------------|--------------------
dft_gibbs_free_energy_ev       |Target| Gibbs free energy of the supramolecular system, calculated using the DFT approximation 
dft_electronic_energy_ev       |Target| Electronic energy of the supramolecular system, calculated using the DFT approximation
dft_entropy_ev       |Target| Entropy of the supramolecular system, calculated using the DFT approximation
dft_enthalpy_ev       |Target| Enthalpy of the supramolecular system, calculated using the DFT approximation
dft_dipole_moment_d       |Target| Dipole moment of the supramolecular system, calculated using the DFT approximation
dft_gap_ev      |Target| Energy gap between HOMO and LUMO, calculated using the DFT approximation
hf_gibbs_free_energy_ev       |Training| Gibbs free energy of the supramolecular system, calculated using the HF approximation 
hf_electronic_energy_ev       |Training| Electronic energy of the supramolecular system, calculated using the HF approximation
hf_entropy_ev       |Training| Entropy of the supramolecular system, calculated using the HF approximation
hf_enthalpy_ev       |Training| Enthalpy of the supramolecular system, calculated using the HF approximation
hf_dipole_moment_d       |Training| Dipole moment of the supramolecular system, calculated using the HF approximation
hf_gap_ev      |Training| Energy gap between HOMO and LUMO, calculated using the HF approximation

In [ ]:
import os
import joblib
import optuna

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns

from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, KFold 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

from scipy import stats
from scipy.stats import pearsonr

# 1. Data Loading 

In [ ]:
#-# Load data from a CSV file
data = pd.read_csv("./NN_ML.csv",delimiter=",",encoding="utf-8")

In [ ]:
#-# Create an empty list to store the '.xyz' coordinates
coordinates_list = []

def read_xyz_file(file_path):
    """
    Function to read XYZ format files and extract atomic coordinates.

    Parameters:
    - file_path (str): Path to the XYZ file.

    Returns:
    - np.array: Numpy array of atomic coordinates.
    """
    with open(file_path, 'r') as file:

        #-# Skip the first two lines containing metadata
        lines = file.readlines()[2:]           
        coordinates = []
        for line in lines:
            parts = line.split()

            #-# Check if the line contains coordinates
            if len(parts) == 4:              
                coordinates.append([float(parts[1]), float(parts[2]), float(parts[3])])
        return np.array(coordinates)

#-# For each row in the DataFrame, form the path to the file and extract coordinates
for index, row in data.iterrows():
    file_name = str(row['name']) + ".xyz"
    file_path = os.path.join("./xyz", file_name)
    coordinates = read_xyz_file(file_path)
    coordinates_list.append(coordinates)

#-# Add the coordinates to the DataFrame as a new column
data['coordinates'] = coordinates_list

# 2. Exploratory Data Analysis (EDA)

### 95% confidence intervals for target (DFT) set

In [ ]:
dft_features = ['dft_gibbs_free_energy_ev', 'dft_electronic_energy_ev', 'dft_entropy_ev',
                'dft_enthalpy_ev', 'dft_dipole_moment_d', 'dft_gap_ev']

metrics = {}

for feature in dft_features:
    mean = data[feature].mean()
    std_dev = data[feature].std()
    n = len(data)
    std_error = std_dev / np.sqrt(n)
    z_score = stats.norm.ppf(0.975) 
    margin_of_error = z_score * std_error
    ci_lower = mean - margin_of_error
    ci_upper = mean + margin_of_error
    ci_width = ci_upper - ci_lower
    
    metrics[feature] = {
        '95% CI Lower': ci_lower,
        '95% CI Upper': ci_upper,
        '95% CI Width': ci_width
    }

metrics_df = pd.DataFrame.from_dict(metrics, orient='index')
print(metrics_df)

### Sactter plots

In [ ]:
hf_features = ['hf_gibbs_free_energy_ev', 'hf_electronic_energy_ev', 'hf_entropy_ev',
               'hf_enthalpy_ev', 'hf_dipole_moment_d', 'hf_gap_ev']

dft_features = ['dft_gibbs_free_energy_ev', 'dft_electronic_energy_ev', 'dft_entropy_ev',
                'dft_enthalpy_ev', 'dft_dipole_moment_d', 'dft_gap_ev']

features_of_interest = hf_features + dft_features

for feature in features_of_interest:
    plt.figure(figsize=(8, 6))
    plt.scatter(data.index, data[feature], alpha=0.6)
    plt.title(f'{feature}')
    plt.xlabel('Index')
    plt.ylabel(f'{feature} Value')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

### Correlation matrix between training (HF) and target (DFT) features

In [ ]:
hf_features = ['hf_gibbs_free_energy_ev', 'hf_electronic_energy_ev', 'hf_entropy_ev',
               'hf_enthalpy_ev', 'hf_dipole_moment_d', 'hf_gap_ev']

dft_features = ['dft_gibbs_free_energy_ev', 'dft_electronic_energy_ev', 'dft_entropy_ev',
                'dft_enthalpy_ev', 'dft_dipole_moment_d', 'dft_gap_ev']

correlation_matrix = data[hf_features + dft_features].corr().loc[hf_features, dft_features]

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
plt.title('Correlation Matrix between HF Features and DFT Targets')
plt.show()

### Average Absolute Correlation of each training (HF) feature with any (DFT) target
### Maximum Absolute Correlation of each training (HF) feature with any (DFT) target

In [ ]:
average_correlations = correlation_matrix.abs().mean(axis=1)
ranked_features_avg = average_correlations.sort_values(ascending=False)
print("\nHF features ranked by average absolute correlation:\n",ranked_features_avg)

max_correlations = correlation_matrix.abs().max(axis=1)
ranked_features_max = max_correlations.sort_values(ascending=False)
print("\nHF features ranked by maximum absolute correlation:\n",ranked_features_max)

### Distribution histograms

In [ ]:
def plot_histograms(features, title):
    """
    Function to plot histograms.

    Parameters:
    - features (list): List of feature names for which histograms are to be plotted.
    - title (str): Title for the entire plot.

    Returns:
    - None

    This function generates subplot histograms for each specified feature, arranging them in a grid.
    It dynamically adjusts the layout based on the number of features for optimal display.
    """
    n_cols = 3
    n_rows = (len(features) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 12))
    fig.suptitle(title, fontsize=20)

    for i, feature in enumerate(features):
        ax = axes[i // n_cols, i % n_cols]
        ax.hist(data[feature], bins=30, edgecolor='k', alpha=0.7)
        ax.set_title(feature)
        ax.set_xlabel('Value')
        ax.set_ylabel('Frequency')

    for i in range(len(features), n_rows * n_cols):
        fig.delaxes(axes.flatten()[i])

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

### Distribution histograms for training (HF) set

In [ ]:
plot_histograms(hf_features, 'Distribution Histograms for HF Features')

### Distribution histograms for target (DFT) set

In [ ]:
plot_histograms(dft_features, 'Distribution Histograms for DFT Targets')

### Box-and-Whiskers diagrams

In [ ]:
def plot_boxplots(features, title):
    """
    Function to create box-and-whiskers diagrams.

    Parameters:
    - features (list): List of feature names for which to create boxplots.
    - title (str): Title for the entire plot.

    Returns:
    - None

    This function generates subplots of boxplots for each specified feature, arranging them in a grid.
    It dynamically adjusts the layout based on the number of features for optimal display.
    """
    n_cols = 3
    n_rows = (len(features) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 12))
    fig.suptitle(title, fontsize=20)

    for i, feature in enumerate(features):
        ax = axes[i // n_cols, i % n_cols]
        ax.boxplot(data[feature].dropna(), vert=True, patch_artist=True)
        ax.set_title(feature)
        ax.set_ylabel('Value')

    for i in range(len(features), n_rows * n_cols):
        fig.delaxes(axes.flatten()[i])

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

### Box-and-Whiskers diagrams for training (HF) set 

In [ ]:
plot_boxplots(hf_features, 'Box-and-Whisker Diagrams for HF Features')

### Box-and-Whiskers diagrams for target (DFT) set 

In [ ]:
plot_boxplots(dft_features, 'Box-and-Whisker Diagrams for DFT Targets')

### P-values for training (HF) and target (DFT) features

In [ ]:
p_values = pd.DataFrame(index=hf_features, columns=dft_features)

for hf in hf_features:
    for dft in dft_features:
        _, p_value = pearsonr(data[hf], data[dft])
        p_values.loc[hf, dft] = p_value

print("\nP-values for correlations between HF features and DFT targets:\n", p_values)

### Outliers

In [ ]:
dataframe = pd.read_csv('./NN_ML.csv')
dataframe = dataframe.drop(columns=['mass_au', 'name'])
dataframe = dataframe.apply(pd.to_numeric, errors='coerce')

def detect_outliers_iqr(data):
    """
    Function to detect outliers in data using the Interquartile Range (IQR).

    Parameters:
    - data (pandas.Series or pandas.DataFrame): One-dimensional or two-dimensional dataset for outlier analysis.

    Returns:
    - outliers (pandas.Series or pandas.DataFrame): Boolean array of the same shape as the input data,
      indicating the presence (True) or absence (False) of outliers in the data.
    """
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR)))
    return outliers

outliers = dataframe.apply(detect_outliers_iqr)
print("\nOutliers detected in each feature:\n", outliers.sum())

sns.pairplot(dataframe[hf_features + dft_features])
plt.suptitle('Pairplot of HF and DFT Features', y=1.02)
plt.show()

# 3. Elastic Net Regression

### Define feature and target sets

In [ ]:
features = ['hf_gibbs_free_energy_ev', 'hf_electronic_energy_ev', 'hf_entropy_ev',
            'hf_enthalpy_ev', 'hf_dipole_moment_d', 'hf_gap_ev', 'coordinates']

targets = ['dft_gibbs_free_energy_ev', 'dft_electronic_energy_ev', 'dft_entropy_ev',
           'dft_enthalpy_ev', 'dft_dipole_moment_d', 'dft_gap_ev']

#-# Create DataFrame for features and target variables
X = data[features].copy()
y = data[targets].copy()

## 3.1 Data normalization

In [ ]:
#-# Flatten the coordinates into 1D arrays
X['coordinates'] = X['coordinates'].apply(np.ravel)

#-# Find the maximum length of the coordinate arrays
max_length = X['coordinates'].apply(len).max()

#-# Pad coordinates with zeros to the maximum length
X['coordinates'] = X['coordinates'].apply(lambda x: np.pad(x, (0, max_length - len(x)), mode='constant'))

#-# Create a new DataFrame with coordinates as individual columns
X_numeric = pd.concat([X.drop(columns='coordinates'),
                       pd.DataFrame(np.vstack(X['coordinates']), 
                                    columns=[f'coord_{i}' for i in range(max_length)], 
                                    index=X.index)],
                      axis=1)

### Split data into training and test sets

In [ ]:
X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(X_numeric, y, test_size=0.1, random_state=42)

#-# Track indices of the test set
test_indices_list = X_test_numeric.index.tolist()

### Normalize the training data

In [ ]:
scaler = StandardScaler()

#-# Save column names for later DataFrame reconstruction
all_columns = X_train_numeric.columns

#-# Apply standardization to the training data
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric[all_columns])

#-# Apply the same transformation to the test data
X_test_numeric_scaled = scaler.transform(X_test_numeric[all_columns])

#-# Reconstruct DataFrame with normalized data for the training set
X_train_numeric_scaled = pd.DataFrame(X_train_numeric_scaled, columns=all_columns, index=X_train_numeric.index)

#-# Reconstruct DataFrame with normalized data for the test set
X_test_numeric_scaled = pd.DataFrame(X_test_numeric_scaled, columns=all_columns, index=X_test_numeric.index)

### Normalize the target values

In [ ]:
scaler_y = StandardScaler()

#-# Apply standardization to the target variables in the training set
y_train_scaled = scaler_y.fit_transform(y_train)

#-# Apply the same transformation to the target variables in the test set
y_test_scaled = scaler_y.transform(y_test)

### Save the scalers

In [ ]:
scaler_dir = './output'
os.makedirs(scaler_dir, exist_ok=True)

joblib.dump(scaler, os.path.join(scaler_dir, 'scaler.pkl'))
joblib.dump(scaler_y, os.path.join(scaler_dir, 'scaler_y.pkl')) 

## 3.2 Architecture of "Elastic Net Regression" model

### 3.2.1 Optimization

### Define the objective function for Optuna optimization

In [ ]:
def objective(trial):
    """
    Objective function for Optuna that defines the task for hyperparameter search.

    Parameters:
    - trial (optuna.trial.Trial): Optuna trial object used to suggest hyperparameters.

    Returns:
    - float: Mean MSE value across all folds of cross-validation.
    """
    #-# Set the hyperparameter grid
    param = {
        'alpha': trial.suggest_float('alpha', 1e-3, 1000.0, log=True),
        'l1_ratio': trial.suggest_float('l1_ratio', 0.0, 1.0),
        'tol': 1e-4,
        'selection': 'cyclic'
    }
    
    #-# Create a model with the current hyperparameter values
    model = ElasticNet(**param, random_state=42, max_iter=100000)
    
    #-# Create a KFold object for 3-fold cross-validation with data shuffling
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    mse_list = []

    #-# Loop through the data splits
    for train_index, val_index in kf.split(X_train_numeric_scaled):
        X_train_cv, X_valid_cv = X_train_numeric_scaled.iloc[train_index], X_train_numeric_scaled.iloc[val_index]
        y_train_cv, y_valid_cv = y_train_scaled[train_index], y_train_scaled[val_index]
        
        #-# Train the model
        model.fit(X_train_cv, y_train_cv)
        
        #-# Evaluate on the validation set
        y_pred_scaled_cv = model.predict(X_valid_cv)
        mse = mean_squared_error(y_valid_cv, y_pred_scaled_cv)
        mse_list.append(mse)
    
    return np.mean(mse_list)

In [ ]:
#-# Run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)

### Save the optimization results

In [ ]:
#-# Print the best hyperparameters
best_trial = study.best_trial
print(f'Best trial value: {best_trial.value}')
print(f'Best hyperparameters: {best_trial.params}')

with open('./optimization_results.txt', 'w') as f:
    f.write(f'Best trial value: {best_trial.value}\n')
    f.write(f'Best hyperparameters: {best_trial.params}\n')

    f.write('\nAll trial results:\n')
    for trial in study.trials:
        f.write(f'Trial {trial.number}: Value={trial.value}, Params={trial.params}\n')

In [ ]:
#-# Get the best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)

### 3.2.2 Training & Validation

In [ ]:
#-# Train the model using the best hyperparameters
best_enr = ElasticNet(**best_params, random_state=42, max_iter=100000)
best_enr.fit(X_train_numeric_scaled, y_train_scaled)

### Save the trained model

In [ ]:
model_dir = "./output"

joblib.dump(best_enr, os.path.join(model_dir, 'elastic_net_model.pkl'))
print(f"Trained model with Optuna saved to {os.path.join(model_dir, 'elastic_net_model.pkl')}")

### 3.2.3 Evaluation

In [ ]:
#-# Predict values on the test data
y_pred_scaled = best_enr.predict(X_test_numeric_scaled)

#-# Inverse transform to get original values
y_pred_inverse = scaler_y.inverse_transform(y_pred_scaled)
y_test_inverse = scaler_y.inverse_transform(y_test_scaled) 

# 4. Visualization

In [ ]:
#-# Create a .csv file with three columns: index, actual value, and predicted value
data = {
    'Index': test_indices_list,
    'Actual_Gibbs_Energy': y_test_inverse[:, 0],
    'Predicted_Gibbs_Energy': y_pred_inverse[:, 0],
    'Actual_Electronic_Energy': y_test_inverse[:, 1],
    'Predicted_Electronic_Energy': y_pred_inverse[:, 1],
    'Actual_Entropy': y_test_inverse[:, 2],
    'Predicted_Entropy': y_pred_inverse[:, 2],
    'Actual_Enthalpy': y_test_inverse[:, 3],
    'Predicted_Enthalpy': y_pred_inverse[:, 3],
    'Actual_Dipole_Moment': y_test_inverse[:, 4],
    'Predicted_Dipole_Moment': y_pred_inverse[:, 4],
    'Actual_Band_Gap': y_test_inverse[:, 5],
    'Predicted_Band_Gap': y_pred_inverse[:, 5]
}

df = pd.DataFrame(data)

#-# Add a column with the color of predicted points (red if observation number < 48)
df['Color'] = df['Index'].apply(lambda x: 'red' if x < 48 else 'blue')

#-# Save the file
csv_path = "./test_results.csv"
df.to_csv(csv_path, index=False)
print(f"Saved DataFrame to {csv_path}")

#-# Set title
target_names = [
    'Gibbs Energy', 'Electronic Energy', 'Entropy', 
    'Enthalpy', 'Dipole Moment', 'Band Gap'
]

#-# Calculate metrics
metrics = {}
for i, target in enumerate(target_names):
    mae = mean_absolute_error(y_test_inverse[:, i], y_pred_inverse[:, i])
    mape = mean_absolute_percentage_error(y_test_inverse[:, i], y_pred_inverse[:, i])
    mse = mean_squared_error(y_test_inverse[:, i], y_pred_inverse[:, i])
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_inverse[:, i], y_pred_inverse[:, i])
    
    metrics[target] = {
        'MSE': mse,
        'RMSE': rmse,
        'R-squared': r2,
        'MAE': mae,
        'MAPE': mape
    }

#-# Create subplots
fig, axs = plt.subplots(2, 3, figsize=(18, 12))
axs = axs.flatten()

for i, target in enumerate(target_names):
    ax = axs[i]
    
    #-# Create scatter plot with conditional coloring
    sns.scatterplot(
        x=df[f'Actual_{target.replace(" ", "_")}'],
        y=df[f'Predicted_{target.replace(" ", "_")}'],
        hue=df['Color'],
        palette={'red': 'red', 'blue': 'blue'},
        ax=ax,
        alpha=0.5,
        legend=False
    )
    
    #-# Create diagonal line
    ax.plot(
        [df[f'Actual_{target.replace(" ", "_")}'].min(), df[f'Actual_{target.replace(" ", "_")}'].max()],
        [df[f'Actual_{target.replace(" ", "_")}'].min(), df[f'Actual_{target.replace(" ", "_")}'].max()],
        color='black', linestyle='--'
    )
    
    #-# Set titles for axes 
    ax.set_xlabel('Actual Values', fontsize=12)
    ax.set_ylabel('Predicted Values', fontsize=12)
    ax.set_title(target, fontsize=14)
    ax.grid(True)
    
    #-# Add metrics to the plots
    ax.text(
        0.05, 0.95,
        f"MSE: {metrics[target]['MSE']:.4f}\nRMSE: {metrics[target]['RMSE']:.4f}\nR-squared: {metrics[target]['R-squared']:.4f}\nMAE: {metrics[target]['MAE']:.4f}\nMAPE: {metrics[target]['MAPE']:.2f}%",
        transform=ax.transAxes, fontsize=12, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    )

for j in range(len(target_names), len(axs)):
    axs[j].axis('off')

plt.tight_layout()
plot_path = "./test_results.png"
plt.savefig(plot_path)
print(f"Saved plots to {plot_path}")